In [10]:
# # torch geometric
# try: 
#     import torch_geometric
# except ModuleNotFoundError:
#     # Installing torch geometric packages with specific CUDA+PyTorch version. 
#     # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details 
#     TORCH = torch.__version__.split('+')[0]
#     CUDA = 'cu' + torch.version.cuda.replace('.','')

#     !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#     !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#     !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#     !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#     !pip install torch-geometric 
#     import torch_geometric
# import torch_geometric.nn as geom_nn
# import torch_geometric.data as geom_data

Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu102.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 5.8 MB/s eta 0:00:00a 0:00:010m
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu102.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 2.7 MB/s eta 0:00:00a 0:00:010m
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu102.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.6 MB/s eta 0:00:00a 0:00:010m
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu102.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 672.7/672.7 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=629404 sha256=43d95ef9eb8320a89e6a895499da434a4143b18753aa8d64b9570b0e3e7b2156
  Stored in directory: /root/.cache/pip/wheels/c

In [ ]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
print("Dataset:", dataset)
print("# Graphs:", len(dataset))
print("# Features:", dataset.num_features)
print("# Classes:", dataset.num_classes)

___

### Get normal model

In [1]:
import torch

from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.models.models import se2d_fullgaussian2d
from nnsysident.training.trainers import standard_trainer as trainer
from nnsysident.utility.measures import get_correlations, get_fraction_oracles
from nnsysident.utility.data_helpers import extract_data_key

device="cuda"

Connecting konstantin@134.76.19.44:3306


In [2]:
paths = ['/notebooks/data/static20457-5-9-preproc0']
data_key = extract_data_key(paths[0])

dataset_config = {'paths': paths, 
                  'batch_size': 64, 
                  'seed': 1, 
                  'cuda': True if device=="cuda" else False,
                  'normalize': True, 
                  'exclude': "images"}

dataloaders = static_loaders(**dataset_config)

In [3]:
model_config = {'init_mu_range': 0.55,
                'init_sigma': 0.4,
                'input_kern': 15,
                'hidden_kern': 13,
                'gamma_input': 1.0,
                'grid_mean_predictor': {'type': 'cortex',
                                        'input_dimensions': 2,
                                        'hidden_layers': 0,
                                        'hidden_features': 0,
                                        'final_tanh': False},
                'gamma_readout': 2.439}

model = se2d_fullgaussian2d(**model_config, dataloaders=dataloaders, seed=1)

/usr/local/lib/python3.8/dist-packages/neuralpredictors/layers/readouts/base.py:72: UserWarning: Use of 'gamma_readout' is deprecated. Please consider using the readout's feature-regularization parameter instead
  warnings.warn(


In [4]:
transfer_model = torch.load('models/transfer_model_with_readout.pth.tar') 
model.load_state_dict(transfer_model, strict=True)
model.to(device).eval();

In [ ]:
# detach_core=True
# if detach_core:
#     print('Core is fixed and will not be fine-tuned')
# else:
#     print('Core will be fine-tuned')

# trainer_config = {'track_training': True,
#                   'detach_core': detach_core}

# score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1, **trainer_config)

In [5]:
for images, responses in dataloaders["train"][data_key]:
    pass

In [8]:
aa = model(images, feature_out=True)
aa.shape

torch.Size([56, 64, 5335, 1])

In [13]:
from torch_geometric.nn import GCNConv


In [14]:
# in_channels=in_channels, hidden_channels=hidden_channels, num_layers=num_layers
g = GCNConv(1,1)

In [15]:
g(aa)

TypeError: forward() missing 1 required positional argument: 'edge_index'